In [1]:
#####欠損値のある非負値行列因子分解#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.linalg
import scipy.stats as ss
import itertools
from numpy.random import *
from scipy import optimize
from scipy.stats import norm
from scipy import sparse

In [2]:
####データの発生####
##データの設定
k = 10   #基底数
hh = 5000   #ユーザー数
item = 3000   #アイテム数
hhpt = hh*item
vec_k = np.repeat(1, k)

In [3]:
##IDとインデックスの設定
#IDの設定
user_id0 = np.repeat(range(hh), item)
item_id0 = np.matlib.repmat(range(item), 1, hh).reshape(-1)

In [4]:
#インデックスの設定
index = np.array(range(hhpt))
user_index0 = [i for i in range(hh)]
item_index0 = [j for j in range(item)]
for i in range(hh):
    user_index0[i]  = index[user_id0==i]
for j in range(item):
    item_index0[j] = index[item_id0==j]

In [5]:
##非負値行列因子分解の仮定に基づきデータを生成
#ガンマ分布よりパラメータを設定
alpha01 = 0.35; beta01 = 1.0
alpha02 = 0.25; beta02 = 0.85
W = np.random.gamma(alpha01, 1/beta01, hh*k).reshape(hh, k)   #ユーザー特徴行列
H = np.random.gamma(alpha02, 1/beta02, item*k).reshape(item, k)   #アイテム特徴行列
WT = W; HT = H
WH = np.dot(W, H.T).reshape(-1)

#購買確率をベータ分布から生成
alpha03 = 9.5; beta03 = 10.0
alpha04 = 7.5; beta04 = 8.0
beta1 = np.random.beta(alpha03, beta03, hh)   #ユーザー購買確率
beta2 = np.random.beta(alpha04, beta04, item)   #アイテム購買確率

#ポアソン分布よりデータを生成
y_comp = np.random.poisson(WH, hhpt)

In [6]:
##欠損のある購買データを生成
#欠損ベクトルを生成
z_vec0 = np.random.binomial(1, beta1[user_id0]*beta2[item_id0], hhpt)
z_vec = z_vec0 * y_comp > 0

#欠損インデックス
index_z0 = index[z_vec0==0]
index_z1 = index[(z_vec0==1) & (y_comp > 0)]
N = index_z1.shape[0]

#購買ベクトルに変換
user_id = user_id0[index_z1]
item_id = item_id0[index_z1]
y_vec = y_comp[index_z1]
y = y_comp[index_z1]   #観測されたデータ

In [7]:
#テストデータの作成
user_id_test = user_id0[index_z0]
item_id_test = item_id0[index_z0]
y_test = y_comp[index_z0]

In [8]:
####マルコフ連鎖モンテカルロ法でImcomplete data NMFを推定####
##アルゴリズムの設定
R = 2000
keep = 2
burnin = int(500/keep)
iter = 0
disp = 10

In [9]:
##ユーザーおよびアイテムのインデックスを作成
#インデックスを作成
index = np.array(range(N))
user_n = np.repeat(0, hh); item_n = np.repeat(0, item)
user_index = [i for i in range(hh)]
item_index = [j for j in range(item)]
for i in range(hh):
    user_index[i]  = index[user_id==i]
    user_n[i] = user_index[i].shape[0]
for j in range(item):
    item_index[j] = index[item_id==j]
    item_n[j] = item_index[j].shape[0]
    
#和を取るためのスパース行列
user_dt = sparse.coo_matrix((np.repeat(1, N), (user_id, range(N))), shape=(hh, N)).tocsr()
item_dt = sparse.coo_matrix((np.repeat(1, N), (item_id, range(N))), shape=(item, N)).tocsr()

In [10]:
#事前分布の設定
alpha1 = 0.1; beta1 = 1
alpha2 = 0.1; beta2 = 1

In [11]:
#パラメータの真値
W = WT
H = HT
WH = np.dot(W[user_id, ] * H[item_id, ], vec_k)

In [12]:
#初期値の設定
W = np.random.gamma(0.1, 1/0.25, hh*k).reshape(hh, k)
H = np.random.gamma(0.1, 1/0.25, item*k).reshape(item, k)
WH = np.dot(W[user_id, ] * H[item_id, ], vec_k)

In [13]:
#サンプリング結果の保存用配列
W_array = np.zeros((hh, k, int(R/keep)))
H_array = np.zeros((item, k, int(R/keep)))

In [14]:
##対数尤度の基準値
#1パラメータモデルの対数尤度
z_st = np.random.binomial(1, 1-np.exp(-np.mean(y)), N)
LLst = np.sum(scipy.stats.poisson.logpmf(z_st*y, np.mean(z_st*y)))
LLst_test = np.sum(scipy.stats.poisson.logpmf(y_test, np.mean(y)))

#真値の対数尤度
z_best = np.random.binomial(1, 1-np.exp(-np.dot(WT[user_id, ] * HT[item_id, ], vec_k)), N)
LLbest = np.sum(scipy.stats.poisson.logpmf(z_best*y, np.dot(WT[user_id, ] * HT[item_id, ], vec_k)))
LLbest_test = np.sum(scipy.stats.poisson.logpmf(y_test, np.dot(WT[user_id_test, ] * HT[item_id_test, ], vec_k)))

In [15]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##潜在変数zをサンプリング
    #応答変数の重み確率を設定
    WH = np.dot(W[user_id, ] * H[item_id, ], vec_k)
    Prob = 1 - np.exp(-WH)

    #潜在変数zから新しい応答変数を設定
    z = np.random.binomial(1, Prob, N)   #ベルヌーイ分布からzをサンプリング
    y_new = z * y


    ##ガンマ分布よりユーザー特徴行列Wをサンプリング
    #補助変数lambdaを更新
    W_vec = W[user_id, ]; H_vec = H[item_id, ]
    WH_vec = W_vec * H_vec
    WH = np.dot(WH_vec, vec_k)   #観測データのNMFの期待値
    Lambda = WH_vec / WH.reshape(N, 1)

    #ユーザーごとのガンマ分布のパラメータ
    lambda_y = Lambda * y_new.reshape(N, 1)
    lambda_h = H[item_id, ]
    W1 = np.zeros((hh, k)); W2 = np.zeros((hh, k))
    for i in range(hh):
        W1[i, ] = np.sum(lambda_y[user_index[i], ], axis=0)
        W2[i, ] = np.sum(lambda_h[user_index[i], ], axis=0)
    W1 = W1 + alpha1; W2 = W2 + beta1

    #パラメータをサンプリング
    W = np.random.gamma(W1.reshape(-1), 1/W2.reshape(-1), hh*k).reshape(hh, k)

    ##ガンマ分布よりアイテム特徴行列Hをサンプリング
    #補助変数lambdaを更新
    W_vec = W[user_id, ]; H_vec = H[item_id, ]
    WH_vec = W_vec * H_vec
    WH = np.dot(WH_vec, vec_k)   #観測データのNMFの期待値
    Lambda = WH_vec / WH.reshape(N, 1)

    #ユーザーごとのガンマ分布のパラメータ
    lambda_y = Lambda * y_new.reshape(N, 1)
    lambda_w = W[user_id, ]
    H1 = np.zeros((item, k)); H2 = np.zeros((item, k))
    for j in range(item):
        H1[j, ] = np.sum(lambda_y[item_index[j], ], axis=0)
        H2[j, ] = np.sum(lambda_w[item_index[j], ], axis=0)
    H1 = H1 + alpha1; H2 = H2 + beta1

    #パラメータをサンプリング
    H = np.random.gamma(H1.reshape(-1), 1/H2.reshape(-1), item*k).reshape(item, k)


    ##パラメータの格納とサンプリング結果の表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        W_array[:, :, mkeep] = W
        H_array[:, :, mkeep] = H

    if rp%disp==0:
        #対数尤度の更新
        LL = np.sum(scipy.stats.poisson.logpmf(y_new, np.dot(W[user_id, ] * H[item_id, ], vec_k)))
        LL_test = np.sum(scipy.stats.poisson.logpmf(y_test, np.dot(W[user_id_test, ] * H[item_id_test, ], vec_k)))

        #サンプリング結果を確認
        print(rp)
        print(np.mean(z))
        print(np.round(np.array([LL, LLst, LLbest]), 2))
        print(np.round(np.array([LL_test, LLst_test, LLbest_test]), 2))        

0
0.3254798971958171
[-1486969.65 -2992733.01 -2250817.58]
[-33352211.46 -22757812.22 -11917228.08]
10
0.5994851252300836
[-2427417.96 -2992733.01 -2250817.58]
[-17428597.23 -22757812.22 -11917228.08]
20
0.6626356562671152
[-2526301.68 -2992733.01 -2250817.58]
[-16194466.11 -22757812.22 -11917228.08]
30
0.6776104492376902
[-2524334.69 -2992733.01 -2250817.58]
[-15822876.68 -22757812.22 -11917228.08]
40
0.6804098749589844
[-2492231.24 -2992733.01 -2250817.58]
[-15470121.07 -22757812.22 -11917228.08]
50
0.6767486652411349
[-2444040.88 -2992733.01 -2250817.58]
[-15038874.33 -22757812.22 -11917228.08]
60
0.6705240598137864
[-2398247.   -2992733.01 -2250817.58]
[-14609785.62 -22757812.22 -11917228.08]
70
0.6653734824246076
[-2361508.98 -2992733.01 -2250817.58]
[-14284436.67 -22757812.22 -11917228.08]
80
0.6609181872407177
[-2335357.59 -2992733.01 -2250817.58]
[-13996895.77 -22757812.22 -11917228.08]
90
0.6550973334097737
[-2309355.52 -2992733.01 -2250817.58]
[-13779159.96 -22757812.22 -1191

810
0.6285698776913216
[-2199623.97 -2992733.01 -2250817.58]
[-12763617.7  -22757812.22 -11917228.08]
820
0.628477173397283
[-2197391.91 -2992733.01 -2250817.58]
[-12779467.74 -22757812.22 -11917228.08]
830
0.6286650215720453
[-2197910.69 -2992733.01 -2250817.58]
[-12781408.19 -22757812.22 -11917228.08]
840
0.6281161145678699
[-2199180.16 -2992733.01 -2250817.58]
[-12775354.4  -22757812.22 -11917228.08]
850
0.6276452743376217
[-2198190.13 -2992733.01 -2250817.58]
[-12781786.87 -22757812.22 -11917228.08]
860
0.6281246531212682
[-2197446.55 -2992733.01 -2250817.58]
[-12787644.99 -22757812.22 -11917228.08]
870
0.6282441928688441
[-2197408.53 -2992733.01 -2250817.58]
[-12782720.83 -22757812.22 -11917228.08]
880
0.6284594863938152
[-2198443.51 -2992733.01 -2250817.58]
[-12772512.94 -22757812.22 -11917228.08]
890
0.6290260804014584
[-2199094.78 -2992733.01 -2250817.58]
[-12783007.79 -22757812.22 -11917228.08]
900
0.6280575644874246
[-2198957.12 -2992733.01 -2250817.58]
[-12778943.58 -2275781

1610
0.6274702339929569
[-2196654.35 -2992733.01 -2250817.58]
[-12788801.18 -22757812.22 -11917228.08]
1620
0.6279050903195981
[-2197378.14 -2992733.01 -2250817.58]
[-12785294.51 -22757812.22 -11917228.08]
1630
0.6287125935124072
[-2199191.05 -2992733.01 -2250817.58]
[-12774130.41 -22757812.22 -11917228.08]
1640
0.6280429269673132
[-2198129.27 -2992733.01 -2250817.58]
[-12772577.79 -22757812.22 -11917228.08]
1650
0.6279209476330521
[-2198418.32 -2992733.01 -2250817.58]
[-12784778.03 -22757812.22 -11917228.08]
1660
0.6281533182648196
[-2197895.45 -2992733.01 -2250817.58]
[-12782790.36 -22757812.22 -11917228.08]
1670
0.6283984967266846
[-2197309.89 -2992733.01 -2250817.58]
[-12784481.53 -22757812.22 -11917228.08]
1680
0.628217967311978
[-2198360.22 -2992733.01 -2250817.58]
[-12787586.7  -22757812.22 -11917228.08]
1690
0.6284296014569212
[-2197426.6  -2992733.01 -2250817.58]
[-12788016.94 -22757812.22 -11917228.08]
1700
0.6280593941774385
[-2197768.88 -2992733.01 -2250817.58]
[-12773818.0